In [ ]:
# Downgrade pytorch and torchvision to get models compatible with aws deployment
!pip install torch==1.5.1+cu92 torchvision==0.6.1+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 604.8MB 27kB/s 
     |████████████████████████████████| 6.5MB 2.0MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown https://drive.google.com/uc?id=17ZuDrk-aqJF5SXZmAwCyNTto3ubi6QWj

Downloading...
From: https://drive.google.com/uc?id=17ZuDrk-aqJF5SXZmAwCyNTto3ubi6QWj
To: /content/cars_data.zip
5.59MB [00:00, 49.0MB/s]


In [ ]:
!unzip -q "/content/cars_data.zip"

In [ ]:
folder = "/content/cars_data"

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
import numpy as np
import datetime
import os, sys

In [ ]:
import glob
from PIL import Image
from matplotlib.pyplot import imshow, imsave
%matplotlib inline
MODEL_NAME = 'DCGAN'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
folder = "/content/cars_data"

In [ ]:
!rm /content/cars_data/.DS_Store

In [ ]:
for filename in os.listdir(folder):
    img = Image.open(os.path.join(folder,filename)).convert('RGB')
    img.save(os.path.join(folder,filename)) 
    

In [ ]:
IMAGE_DIM = (96, 96, 3)

In [ ]:
def get_sample_image(G, n_noise):
    """
        save sample 100 images
    """
    z = torch.randn(10, n_noise).to(DEVICE)
    y_hat = G(z).view(10, 3, 96, 96).permute(0, 2, 3, 1) 
    result = (y_hat.detach().cpu().numpy()+1)/2.
    return result

In [ ]:
def get_sample_image_stack(G, n_noise=100,n_rows=3,n_cols=3):
  n_imgs= n_rows*n_cols
  z = torch.randn(n_imgs, n_noise).to(DEVICE)
  y_hat = G(z).view(n_imgs, 3, 96, 96).permute(0, 2, 3, 1) # (100, 28, 28)
  result = (y_hat.detach().cpu().numpy()+1)/2.
  # now stack up the images column wise for each rows
  img_stack = [np.concatenate(result[i*n_cols:i*n_cols+n_cols],1) for i in range(n_rows) ]
  #print(img_stack[0].shape)
  img_stack = np.concatenate(img_stack,0)
  #print(img_stack.shape)
  return img_stack

In [ ]:
class Discriminator(nn.Module):
    """
        Convolutional Discriminator for MNIST
    """
    def __init__(self, in_channel=1, num_classes=1):
        super(Discriminator, self).__init__()
        self.conv = nn.Sequential(
            # 100 -> 50
            nn.Conv2d(in_channel, 512, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            # 50 -> 25
            nn.Conv2d(512, 512, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            # 25 -> 13
            nn.Conv2d(512, 256, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            # 13 -> 7
            nn.Conv2d(256, 128, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            # 7 -> 4
            nn.Conv2d(128, 128, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1),
        )        
        self.fc = nn.Sequential(
            # reshape input, 128 -> 1
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x, y=None):
        y_ = self.conv(x)
        y_ = y_.view(y_.size(0), -1)
        y_ = self.fc(y_)
        return y_

In [ ]:
class Generator(nn.Module):
    """
        Convolutional Generator for MNIST
    """
    def __init__(self, out_channel=1, input_size=100, num_classes=784):
        super(Generator, self).__init__()
        assert IMAGE_DIM[0] % 2**5 == 0, 'Should be divided 32' # we want to generate our image in 5 steps as in Descriminator
        self.init_dim = (IMAGE_DIM[0] // 2**5, IMAGE_DIM[1] // 2**5)
        self.fc = nn.Sequential(
            nn.Linear(input_size, self.init_dim[0]*self.init_dim[1]*512),    # 1*100 -> 3*3*512
            nn.ReLU(),
        )
        self.conv = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1, bias=False),                  # 512
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # x2
            nn.ConvTranspose2d(512, 512, 4, stride=2, padding=1, bias=False),  # 512x6x6
            nn.BatchNorm2d(512),
            nn.ReLU(),            
            # x2
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1, bias=False),   # 256x12x12
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # x2
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1, bias=False),   # 128x24x24
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # x2
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1, bias=False),   # 128x48x48
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # x2
            nn.ConvTranspose2d(128, out_channel, 4, stride=2, padding=1, bias=False), # 3x96x96
            nn.Tanh(),
        )
        
    def forward(self, x, y=None):
        x = x.view(x.size(0), -1)
        y_ = self.fc(x)
        y_ = y_.view(y_.size(0), 512, self.init_dim[0], self.init_dim[1])
        y_ = self.conv(y_)
        return y_

In [ ]:
class CARS(Dataset):
    '''
    CARS Dataset
    You should download this dataset from below url.
    url: https://ai.stanford.edu/~jkrause/cars/car_dataset.html
    '''
    def __init__(self, data_path, transform=None):
        '''
        Args:
            data_path (str): path to dataset
        '''
        self.data_path = data_path
        self.transform = transform
        self.fpaths = sorted(glob.glob(os.path.join(data_path, '*.png')))
        #gray_lst = [266, 1085, 2176, 3048, 3439, 3469, 3539, 4577, 4848, 5177, 5502, 5713, 6947, 7383, 7693, 7774, 8137, 8144]
        #for num in gray_lst:
        #    self.fpaths.remove(os.path.join(data_path, '{:05d}.png'.format(num)))
        
    def __getitem__(self, idx):
        img = self.transform(Image.open(self.fpaths[idx]))
        return img

    def __len__(self):
        return len(self.fpaths)

In [ ]:
from torchsummary import summary
IMAGE_DIM = (96, 96, 3)
D = Discriminator(in_channel=IMAGE_DIM[-1]).to(DEVICE)
summary(D, input_size=(3, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 512, 48, 48]          13,824
       BatchNorm2d-2          [-1, 512, 48, 48]           1,024
         LeakyReLU-3          [-1, 512, 48, 48]               0
            Conv2d-4          [-1, 512, 24, 24]       2,359,296
       BatchNorm2d-5          [-1, 512, 24, 24]           1,024
         LeakyReLU-6          [-1, 512, 24, 24]               0
            Conv2d-7          [-1, 256, 12, 12]       1,179,648
       BatchNorm2d-8          [-1, 256, 12, 12]             512
         LeakyReLU-9          [-1, 256, 12, 12]               0
           Conv2d-10            [-1, 128, 6, 6]         294,912
      BatchNorm2d-11            [-1, 128, 6, 6]             256
        LeakyReLU-12            [-1, 128, 6, 6]               0
           Conv2d-13            [-1, 128, 3, 3]         147,456
      BatchNorm2d-14            [-1, 12

In [ ]:
from torchsummary import summary
IMAGE_DIM = (96, 96, 3)
G = Generator(out_channel=IMAGE_DIM[-1]).to(DEVICE)
summary(G, input_size=(1, 100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 4608]         465,408
              ReLU-2                 [-1, 4608]               0
            Conv2d-3            [-1, 512, 3, 3]       2,359,296
       BatchNorm2d-4            [-1, 512, 3, 3]           1,024
              ReLU-5            [-1, 512, 3, 3]               0
   ConvTranspose2d-6            [-1, 512, 6, 6]       4,194,304
       BatchNorm2d-7            [-1, 512, 6, 6]           1,024
              ReLU-8            [-1, 512, 6, 6]               0
   ConvTranspose2d-9          [-1, 256, 12, 12]       2,097,152
      BatchNorm2d-10          [-1, 256, 12, 12]             512
             ReLU-11          [-1, 256, 12, 12]               0
  ConvTranspose2d-12          [-1, 128, 24, 24]         524,288
      BatchNorm2d-13          [-1, 128, 24, 24]             256
             ReLU-14          [-1, 128,

In [ ]:

transform = transforms.Compose([transforms.Resize((IMAGE_DIM[0],IMAGE_DIM[1])),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))
                               ]
)

In [ ]:
dataset = CARS(data_path='/content/cars_data/', transform=transform)

In [ ]:
len(dataset)

612

In [ ]:
batch_size = 64

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=8)

In [ ]:
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr=0.001, betas=(0.5, 0.999))
G_opt = torch.optim.Adam(G.parameters(), lr=0.001, betas=(0.5, 0.999))

In [ ]:
max_epoch = 2200
step = 0
n_critic = 1 # for training more k steps about Discriminator
n_noise = 100

In [ ]:
D_labels = torch.ones([batch_size, 1]).to(DEVICE) # Discriminator Label to real
D_fakes = torch.zeros([batch_size, 1]).to(DEVICE) # Discriminator Label to fake

In [ ]:
!mkdir /content/drive/My\ Drive/e4p2_GAN/samples

mkdir: cannot create directory ‘/content/drive/My Drive/e4p2_GAN/samples’: File exists


In [ ]:
from tqdm import tqdm
D_loss_log = []
G_loss_log = []
for epoch in range(max_epoch):
    for idx, images in enumerate(tqdm(data_loader)):
        # Training Discriminator
        x = images.to(DEVICE)
        x_outputs = D(x)
        D_x_loss = criterion(x_outputs, D_labels)

        z = torch.randn(batch_size, n_noise).to(DEVICE)
        z_outputs = D(G(z))
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss
        
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()
        D_loss_log.append(D_loss.item())

        if step % n_critic == 0:
            # Training Generator
            z = torch.randn(batch_size, n_noise).to(DEVICE)
            z_outputs = D(G(z))
            G_loss = criterion(z_outputs, D_labels)
            

            D.zero_grad()
            G.zero_grad()
            G_loss.backward()
            G_opt.step()
            G_loss_log.append(G_loss.item())
        
        if step % 300 == 0:
            dt = datetime.datetime.now().strftime('%H:%M:%S')
            
            G.eval()
            img = get_sample_image_stack(G, n_noise)
            print()
            print('Saving image Step:{}'.format(step))
            imsave('/content/drive/My Drive/e4p2_GAN/samples/{}_step{:05d}.jpg'.format(MODEL_NAME, step), img)
            G.train()
        step += 1
    print()
    print('Epoch: {}/{}, D Loss: {:.4f}, G Loss: {:.4f}, Time:{}'.format(epoch, max_epoch, D_loss.item(), G_loss.item(), dt))

 11%|█         | 1/9 [00:01<00:12,  1.54s/it]


Saving image Step:0


  0%|          | 0/9 [00:00<?, ?it/s]


Epoch: 0/2200, D Loss: 1.2575, G Loss: 0.8704, Time:05:06:33


  0%|          | 0/9 [00:00<?, ?it/s]

...


Epoch: 2197/2200, D Loss: 0.0076, G Loss: 6.5723, Time:08:12:08


  0%|          | 0/9 [00:00<?, ?it/s]


Epoch: 2198/2200, D Loss: 0.0391, G Loss: 5.8116, Time:08:12:08


100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


Epoch: 2199/2200, D Loss: 0.0992, G Loss: 4.0841, Time:08:12:08


In [ ]:
def save_checkpoint(state, file_name='checkpoint.pth.tar'):
    torch.save(state, file_name)

In [ ]:

# Saving params.
# torch.save(D.state_dict(), 'D_c.pkl')
# torch.save(G.state_dict(), 'G_c.pkl')
save_checkpoint({'epoch': epoch + 1,
                 'D':D.state_dict(),
                 'G':G.state_dict(),
                 'd_optim': D_opt.state_dict(),
                 'g_optim' : G_opt.state_dict()},
                '/content/drive/My Drive/e4p2_GAN/models/dcgan.pth.tar')

In [ ]:
traced_G_model = torch.jit.trace(G.to('cpu'), torch.randn(1, 100))

In [ ]:
traced_G_model.save('/content/drive/My Drive/e4p2_GAN/models/Car_Generator.pt')

In [ ]:
traced_D_model = torch.jit.trace(D.to('cpu'), torch.randn(1, 3,96,96))

In [ ]:
traced_D_model.save('/content/drive/My Drive/e4p2_GAN/models/Car_Descriminator.pt')

Generate GIF file


In [ ]:
output_folder = '/content/drive/My Drive/e4p2_GAN/samples/'
listImageNames = glob.glob(os.path.join(output_folder, '*.jpg'))

In [ ]:
list_images = [Image.open(imgName) for imgName in listImageNames]
list_images[0].save('/content/drive/My Drive/e4p2_GAN/modelOutput.gif',
               save_all=True, append_images=list_images[1:], optimize=False, duration=500, loop=0)